In [2]:
import requests_html
from requests_html import HTMLSession
import pandas as pd
from urllib import request

In [3]:
# import brawlers
brawlers = pd.read_csv('datasets/brawlers/brawlers.csv', index_col=0)
brawlers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64 entries, 0 to 63
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          64 non-null     int64 
 1   name        64 non-null     object
 2   starPowers  64 non-null     object
 3   gadgets     64 non-null     object
dtypes: int64(1), object(3)
memory usage: 2.5+ KB


In [71]:
# import brawler avatar icons
for i, brawler in enumerate(brawlers['name']):
	brawler = brawler.lower().replace(' ','_').replace('.','_')
	url = 'https://media.brawltime.ninja/brawlers/' + brawler + '/avatar.webp'
	local = 'resources/brawler_icons/avatar/' + brawler + '.png'
	opener=request.build_opener()
	opener.addheaders=[('User-Agent','Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1941.0 Safari/537.36')]
	request.install_opener(opener)
	request.urlretrieve(url,local)

In [72]:
# import brawler models
for i, brawler in enumerate(brawlers['name']):
	brawler = brawler.lower().replace(' ','_').replace('.','_')
	url = 'https://media.brawltime.ninja/brawlers/' + brawler + '/model.webp'
	local = 'resources/brawler_icons/model/' + brawler + '.png'
	opener=request.build_opener()
	opener.addheaders=[('User-Agent','Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1941.0 Safari/537.36')]
	request.install_opener(opener)
	request.urlretrieve(url,local)

In [ ]:
# pendiente importar maps brawlball, hotzone, gemgrab, knockout

In [ ]:
#SELENIUM
# driver = webdriver.Chrome()

# url = 'https://brawltime.ninja/es/tier-list/brawler/8-bit'

# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')
# driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)


# health=[]
# brawlerRarity=[]
# # content = driver.page_source
# # soup = BeautifulSoup(content)

# driver.get(url.format(s='Pets'))

# print(f'Found {len(get_videos(driver))} boxes.')

# for a in soup.findAll('dl', attrs={'class':'flex flex-col gap-y-1' 'data-v-62f27b4e'}):
#     name=a.find('dt', attrs={'class':'basis-[min-content] flex-grow text-text/75'})
#     brawlerRarity=a.find('dd', attrs={'class':'basis-[auto] text-text text-right'})
#     health.append(name)

# print(name)

In [10]:
# webscraping brawlers with requests_html
indicators = []

for i, brawler in enumerate(brawlers['name']):	
	brawler = brawler.lower().replace(' ','_').replace('.','_')
	session = HTMLSession()
	url = 'https://brawltime.ninja/tier-list/brawler/' + brawler
	response = session.get(url)

	titles = []
	contents = []

	# list = response.html.find('div.dashboard__cell', first=True).find('dt')
	list = response.html.find('dt')
	for item in list:
		tmp = item.text.split('\n')
		titles.append(tmp[0])
		if tmp[0] == 'Reload':
			break
		
	list = response.html.find('dd')
	for item in list:
		tmp = item.text.split('\n')
		contents.append(tmp[0])

	clist = dict(zip(titles, contents))
	clist['Brawler'] = brawler
	indicators.append(clist)

In [65]:
# depuración dataframe indicators_df
indicators_df = pd.DataFrame(indicators)

indicators_df = indicators_df[['Brawler','Health','Rarity','Class','Movement Speed','Range','Reload']]

for i, item in enumerate(indicators_df['Health']):
	indicators_df.loc[i, 'Health'] = int(item) * 1.5

for i, item in enumerate(indicators_df['Movement Speed']):
	indicators_df.loc[i, 'Speed_Num'] = str(item).split(sep=' ',maxsplit=1)[0]
	
for i, item in enumerate(indicators_df['Movement Speed']):
	indicators_df.loc[i, 'Movement Speed'] = str(item).split(sep=' ',maxsplit=1)[1].replace('(','').split(sep=')',maxsplit=1)[0]

for i, item in enumerate(indicators_df['Range']):
	indicators_df.loc[i, 'Range_Num'] = str(item).split(sep=' ',maxsplit=1)[0]
	
for i, item in enumerate(indicators_df['Range']):
	try:
		indicators_df.loc[i, 'Range'] = str(item).split(sep=' ',maxsplit=1)[1].replace('(','').split(sep=')',maxsplit=1)[0]
	except:
		indicators_df.loc[i, 'Range'] = ''

for i, item in enumerate(indicators_df['Reload']):
	indicators_df.loc[i, 'Reload_Num'] = str(item).split(sep=' ',maxsplit=1)[0]
	
for i, item in enumerate(indicators_df['Reload']):
	try:
		indicators_df.loc[i, 'Reload'] = str(item).split(sep=' ',maxsplit=2)[2].replace('(','').split(sep=')',maxsplit=1)[0]
	except:
		indicators_df.loc[i, 'Reload'] = ''

roles = pd.read_csv('datasets/brawlers/brawlers_roles.csv', index_col=0)

indicators_df = pd.merge(indicators_df, roles, on='Brawler')

indicators_df.to_csv('datasets/brawlers/brawlers_classification.csv')
indicators_df

,Brawler,Health,Rarity,Class,Movement Speed,Range,Reload,Speed_Num,Range_Num,Reload_Num,id,Role
0,shelly,5700.0,Starting Brawler,Damage Dealer,Normal,Long,Normal,720,7.67,1.5,16000000,tank stopper
1,colt,4200.0,Rare,Damage Dealer,Normal,Long,Normal,720,9,1.7,16000001,ranged dps
2,bull,7500.0,Rare,Tank,Fast,Normal,Normal,770,5.33,1.6,16000002,close quarters
3,brock,3600.0,Rare,Damage Dealer,Normal,Long,Slow,720,9,2.1,16000003,sniper
4,rico,4200.0,Super Rare,Damage Dealer,Normal,Very Long,Very Fast,720,9.67,1.2,16000004,ranged dps
...,...,...,...,...,...,...,...,...,...,...,...,...
59,gus,4800.0,Super Rare,Support,Normal,Very Long,Normal,720,9.33,1.5,16000061,healer
60,buster,7200.0,Chromatic,Tank,Fast,Normal,Normal,770,5.33,1.8,16000062,area control
61,chester,4950.0,Legendary,Damage Dealer,Fast,Long,Normal,770,8.33,1.9,16000063,assassin
62,gray,4950.0,Mythic,Support,Normal,Very Long,Fast,720,9,1.4,16000064,sniper


In [3]:
maplist = pd.read_csv('datasets/maps/maplist.csv', index_col=0)
maplist.columns=['modo','mapa']

modos_hom = pd.DataFrame.from_dict({'knockout': 'knockout','hotZone': 'hot-zone','heist': 'heist','gemGrab': 'gem-grab','brawlBall': 'brawl-ball','bounty': 'bounty'}, orient='index').reset_index()
modos_hom.columns=['modo','modo_hom']

maplist = maplist.merge(modos_hom, on='modo')

for i, mapa in enumerate(maplist['mapa']):
	maplist.loc[i, 'mapa_hom'] = mapa.replace(' ','-')

maplist

,modo,mapa,modo_hom,mapa_hom
0,knockout,X Marks the Spot,knockout,X-Marks-the-Spot
1,knockout,Riverside,knockout,Riverside
2,knockout,Out in the Open,knockout,Out-in-the-Open
3,knockout,New Perspective,knockout,New-Perspective
4,knockout,Goldarm Gulch,knockout,Goldarm-Gulch
5,knockout,Flaring Phoenix,knockout,Flaring-Phoenix
6,knockout,Belle's Rock,knockout,Belle's-Rock
7,hotZone,Split,hot-zone,Split
8,hotZone,Ring of Fire,hot-zone,Ring-of-Fire
9,hotZone,Quarter Pounder,hot-zone,Quarter-Pounder


In [5]:
# import listado de mapas mas url
finalList = []

for i, point in maplist.iterrows():
	
	pairlist=[]

	session = HTMLSession()
	url = 'https://brawltime.ninja/tier-list/mode/' + point.modo_hom + '/map/' + point.mapa_hom
	response = session.get(url)
	#response.html.find('div.dashboard__cell', first=True).find('dt')

	mapname = []
	content = []

	list = response.html.find('img')
	try:
		for item in list:
			if 'alt=' in str(item):
				item = str(item).replace("Belle's",'Belle`s').replace('"',"'")
				tmp = item.split("alt='")[1].split("'")[0]
				tmp = tmp.replace("Belle`s","Belle's")
				mapname.append(tmp)
			else:
				mapname.append('')
	except:
		print("error al importar mapname")

	try:
		for item in list:
			if 'src=' in str(item):
				item = str(item)
				tmp = item.split("src='")[1].split("'")[0]
				tmp = tmp.replace('?size=512','')
				content.append(tmp)
			else:
				content.append('')
	except:
		print("error al importar contenido")
		
	clist = dict(zip(mapname, content))
	finalList.append({point.mapa: clist[point.mapa]})

finalList

[{'X Marks the Spot': 'https://media.brawltime.ninja/maps/15000529.png'},
 {'Riverside': 'https://media.brawltime.ninja/maps/15000501.png'},
 {'Out in the Open': 'https://media.brawltime.ninja/maps/15000548.png'},
 {'New Perspective': 'https://media.brawltime.ninja/maps/15000528.png'},
 {'Goldarm Gulch': 'https://media.brawltime.ninja/maps/15000367.png'},
 {'Flaring Phoenix': 'https://media.brawltime.ninja/maps/15000440.png'},
 {"Belle's Rock": 'https://media.brawltime.ninja/maps/15000368.png'},
 {'Split': 'https://media.brawltime.ninja/maps/15000294.png'},
 {'Ring of Fire': 'https://media.brawltime.ninja/maps/15000300.png'},
 {'Quarter Pounder': 'https://media.brawltime.ninja/maps/15000245.png'},
 {'Parallel Plays': 'https://media.brawltime.ninja/maps/15000293.png'},
 {'Open Zone': 'https://media.brawltime.ninja/maps/15000527.png'},
 {'Dueling Beetles': 'https://media.brawltime.ninja/maps/15000306.png'},
 {'Controller Chaos': 'https://media.brawltime.ninja/maps/15000249.png'},
 {'Torn

In [6]:
# import map models
for i, mapa in enumerate(maplist['mapa']):
	url = [item[mapa] for item in finalList if mapa in item][0]
	local = 'resources/maps/' + mapa + '.png'
	opener=request.build_opener()
	opener.addheaders=[('User-Agent','Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1941.0 Safari/537.36')]
	request.install_opener(opener)
	request.urlretrieve(url,local)
